In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct

In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 1,
    "exp_name": ct.generate_readable_short_id("track_time_test"), #"main_run5_cv5_21data_100trees_40opt_fast",
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 5,
    "plot": False,

    # "calib_methods": ["RF_d", "RF_opt", "RF_large",
    #                   "Platt", "ISO", "Beta", "CRF", "VA",
    #                   "tlr", "Rank", "Elkan",                      
    #                   ],

    "calib_methods": ["RF_opt", "RF_large",
                      "DT", "LR", "SVM", "NN", "GNB"          
                      ],

    "metrics": ["acc", "brier", "ece", "logloss", "time"],

    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth":  np.arange(2, 100).tolist(), # [None, 5, 10, 15, 20, 30, 50], #
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": [2, 5, 10],
                    "min_samples_leaf":  [1, 2, 4],
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false

    "exp_key": "data_name",
    "exp_values": [
                  # "cm1",
                  # "datatrieve",
                  # "kc1_class_level_defectiveornot",
                  "kc1",
                  "kc2",
                  "kc3",
                  # "pc1",
                  # "spect",
                  # "spectf",
                  # "vertebral", 
                #   "wilt",
                #   "parkinsons", 
                #   "heart",
                #   "wdbc",
                #   "bank", 
                #   "ionosphere", 
                #   "HRCompetencyScores",
                #   "spambase", 
                #   "QSAR", 
                #   "diabetes", 
                #   "breast", 
                #   "SPF",
                #   "hillvalley",
                #   "pc4",
                #   "scene",
                #   "Sonar_Mine_Rock_Data",
                #   "Customer_Churn",
                #   "jm1",
                #   "eeg",
                #   "phoneme",


                  # "madelon",
                  # "nomao",
                  ]
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param kc1 done
exp_param kc2 done
exp_param kc3 done


In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)
ct.save_metrics_to_csv(tables, params["metrics"], params['exp_name'])

In [8]:
tables["acc"].round(4)


,RF_large,DT,GNB
Data,,,
kc1,0.8563,0.8521,0.8502
kc2,0.8313,0.8122,0.8161
kc3,0.8930,0.8997,0.9018
Mean,0.8602,0.8546,0.8560
Rank,1.6667,2.3333,2.0000


In [6]:
ct.res_statistics(tables, params["metrics"], f"results/{params['exp_name']}")

metric acc
Friedman Test Statistic: 0.6666666666666643
P-value: 0.71653131057379
The differences between groups are significant.
metric brier
Friedman Test Statistic: 6.0
P-value: 0.04978706836786395
The differences between groups are significant.
metric ece
Friedman Test Statistic: 6.0
P-value: 0.04978706836786395
The differences between groups are significant.
metric logloss
Friedman Test Statistic: 0.6666666666666643
P-value: 0.71653131057379
The differences between groups are significant.
metric time
Friedman Test Statistic: 6.0
P-value: 0.04978706836786395
The differences between groups are significant.
